In [3]:
import torch
import torch.nn as nn


tensor([1., 2., 3.], dtype=torch.float64)

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (self.head_dim*self.heads == self.embed_size), "Embed size not div by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, queries, mask):
        N = queries.shape[0]
        # shape querys = N, query_len(sentence length, words), embed_size 
        # shape keys = N, key_len(sentence length, words), embed_size 
        # shape values = N, value_len(sentence length, words), embed_size 
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, value_len, self.heads, self.head_dim)
        queries = queries.reshape(N, value_len, self.heads, self.head_dim)

        # dot product (energy), the quesry matrix is multiplied by transpose of key matrix
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask==0, float("-1e20")) # using very low number to denote negative inf
        
        attention = torch.softmax(energy/(self.embed_size**0.5), dim=3)

        out = torch.einsum("nvhd,nhqk")
